Install packages

In [2]:
!pip install folium
#!pip3 install KMeans

     |████████████████████████████████| 93 kB 2.2 MB/s eta 0:00:011


Import default tools

In [3]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import folium

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Beautifulsoap
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


1. Download table from wikipedia

In [4]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'html.parser')
My_table = soup.find('table',{'class':'wikitable sortable'})
lt = pd.read_html(str(My_table))
df = lt[0]
df = df.rename(columns={"Neighbourhood": "Neighborhood"})
df.head()
df1 = df[df.Borough != 'Not assigned']
#df1[df1.Neighborhood == 'Not assigned']

2. Download Geodata

In [5]:
url = "http://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)
df2 = df1.join(geodf.set_index('Postal Code'), on='Postal Code')
df3 = df2[df2['Borough'].str.contains('Toronto')]

toronto_data = df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


3. Credential information for using foursquare

In [6]:
# type your answer here
CLIENT_ID = 'UFMC4P3J35E05A0KYJJW0KR455IBIXSCHHSZRICLJZGOZ4FW' # your Foursquare ID
CLIENT_SECRET = 'QIBETEZDQHSLZMJ2DGBQHCB01Q5VCKXDE4YQYSW0BNWVVKCM' # your Foursquare Secret
ACCESS_TOKEN = 'MQX3OFQXNV5VAIUQFVIV0KWD3E05APR02HBF3PK4JQEKH11B' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFMC4P3J35E05A0KYJJW0KR455IBIXSCHHSZRICLJZGOZ4FW
CLIENT_SECRET:QIBETEZDQHSLZMJ2DGBQHCB01Q5VCKXDE4YQYSW0BNWVVKCM


4. Define function for foursquare API

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name, lat, lng)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

5. Create toronto venue dataframe

In [8]:
toronto_venues = getNearbyVenues(toronto_data.Neighborhood, toronto_data.Latitude, toronto_data.Longitude)
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Pharmacy,Farmers Market,Cheese Shop,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant,Burrito Place
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Harbor / Marina
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant


6. Create clusters

In [9]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

#k=5
#toronto_clustering = toronto_data.drop(['Postal Code','Borough','Neighborhood'],1)
#kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
#kmeans.labels_
#toronto_data.insert(0, 'Cluster Labels', kmeans.labels_)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Park,Café,Bakery,Theater,Breakfast Spot,Brewery,Spa,Beer Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,College Cafeteria,Diner,Bar,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Lingerie Store,Japanese Restaurant,Hotel,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Gastropub,Clothing Store,Cocktail Bar,Cosmetics Shop,American Restaurant,Creperie,Japanese Restaurant,Seafood Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Health Food Store,Pub,Wine Bar,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


7. Create visual map

In [10]:
address = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = neighborhood_latitude
longitude = neighborhood_longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        #color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [11]:
## create map
#map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)
#
## set color scheme for the clusters
#x = np.arange(k)
#ys = [i + x + (i*x)**2 for i in range(k)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
#
## add markers to the map
#markers_colors = []
#for lat, lon, neighbourhood, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood'], toronto_data['Cluster Labels']):
#    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
#    folium.CircleMarker(
#        [lat, lon],
#       radius=5,
#        popup=label,
#        color=rainbow[cluster-1],
#        fill=True,
#        fill_color=rainbow[cluster-1],
#        fill_opacity=0.7).add_to(map_clusters)
#       
#map_clusters

Get Tokyo wards table from Wikipedia

In [54]:
website_url = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text

soup = BeautifulSoup(website_url,'html.parser')
My_table = soup.find('table',{'class':'wikitable sortable'})
lt = pd.read_html(str(My_table))
df = lt[0]
df = df.drop(23, axis=0)
df.head()

Create Tokyo_data dataframe

In [75]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
tokyo_data = pd.DataFrame(columns=column_names)
for index in df.index:
    Neighbors = [x.strip() for x in df.at[index,'Major districts'].split(',')]
    for nei in Neighbors:
        #print(df.at[index,'Name'], nei, geolocator.geocode(nei).latitude, geolocator.geocode(nei).longitude)
        tokyo_data = tokyo_data.append({'Borough': df.at[index,'Name'], 'Neighborhood': nei, 'Latitude': geolocator.geocode(nei).latitude, 'Longitude': geolocator.geocode(nei).longitude}, ignore_index=True)


In [76]:
tokyo_data

,Borough,Neighborhood,Latitude,Longitude
0,Bunkyō,Mita,3.500009,-73.000009
1,Chiyoda,Nagatachō,35.675618,139.743469
2,Chiyoda,Kasumigaseki,35.674054,139.750972
3,Chiyoda,Ōtemachi,35.686779,139.763564
4,Chiyoda,Marunouchi,35.680656,139.765222
5,Chiyoda,Akihabara,35.699736,139.771250
6,Chiyoda,Yūrakuchō,35.674423,139.761674
7,Chiyoda,Iidabashi,35.701374,139.744445
8,Chiyoda,Kanda,29.845427,79.936388
9,Chūō,Nihonbashi,35.684068,139.774503


In [77]:
tokyo_venues = getNearbyVenues(tokyo_data.Neighborhood, tokyo_data.Latitude, tokyo_data.Longitude)
# one hot encoding
tokyo_onehot = pd.get_dummies(tokyo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tokyo_onehot['Neighborhood'] = tokyo_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tokyo_onehot.columns[-1]] + list(tokyo_onehot.columns[:-1])
tokyo_onehot = tokyo_onehot[fixed_columns]

#tokyo_onehot.head()

tokyo_grouped = tokyo_onehot.groupby('Neighborhood').mean().reset_index()
#tokyo_grouped

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tokyo_grouped['Neighborhood']

for ind in np.arange(tokyo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tokyo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akabane,Sake Bar,Convenience Store,Ramen Restaurant,BBQ Joint,Soba Restaurant,Café,Clothing Store,Coffee Shop,Italian Restaurant,Supermarket
1,Akihabara,Café,Hobby Shop,Electronics Store,Ramen Restaurant,Arcade,Donburi Restaurant,Rock Club,Toy / Game Store,Comic Shop,BBQ Joint
2,Aobadai,Grocery Store,Ramen Restaurant,Park,Convenience Store,Clothing Store,BBQ Joint,Coffee Shop,Café,Soba Restaurant,Bakery
3,Aomi,Exhibit,Plaza,Convenience Store,Coffee Shop,Park,Bookstore,Italian Restaurant,Theme Restaurant,Theme Park Ride / Attraction,Theme Park
4,Aoto,Convenience Store,Concert Hall,Japanese Restaurant,Chinese Restaurant,Sake Bar,Rental Car Location,Ramen Restaurant,Italian Restaurant,Fried Chicken Joint,BBQ Joint


In [81]:
# set number of clusters
kclusters = 5

tokyo_grouped_clustering = tokyo_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tokyo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tokyo_merged = tokyo_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
tokyo_merged = tokyo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
tokyo_merged = tokyo_merged.dropna()
tokyo_merged.head()

#k=5
#tokyo_clustering = tokyo_data.drop(['Postal Code','Borough','Neighborhood'],1)
#kmeans = KMeans(n_clusters = k,random_state=0).fit(tokyo_clustering)
#kmeans.labels_
#tokyo_data.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [85]:
address = 'Tokyo'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tokyo_merged['Latitude'], tokyo_merged['Longitude'], tokyo_merged['Neighborhood'], tokyo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        #color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Get New York data

In [86]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [88]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [89]:
neighborhoods_data = newyork_data['features']

In [90]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [91]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [92]:
address = 'New York City, NY'
#address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [93]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [94]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [101]:
manhattan_venues = getNearbyVenues(manhattan_data.Neighborhood, manhattan_data.Latitude, manhattan_data.Longitude)
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

#manhattan_onehot.head()

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
#manhattan_grouped

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
mhn_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
mhn_neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    mhn_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

mhn_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Boat or Ferry,Gym,Clothing Store,Memorial Site,Shopping Mall,Sandwich Place,Beer Garden
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Wine Shop,Bar,Yoga Studio,Gym,Cosmetics Shop,French Restaurant,Bookstore
2,Central Harlem,Seafood Restaurant,African Restaurant,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Bar,Bookstore,Park,Beer Bar
3,Chelsea,Coffee Shop,Bakery,Art Gallery,American Restaurant,Hotel,Italian Restaurant,French Restaurant,Wine Shop,Café,Nightclub
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Salon / Barbershop,Spa,Optical Shop,Dessert Shop,Greek Restaurant,Sandwich Place


In [105]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
mhn_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
mhn_kmeans.labels_[0:10]

# add clustering labels
mhn_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', mhn_kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(mhn_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged = manhattan_merged.dropna()
manhattan_merged.head()

#k=5
#manhattan_clustering = manhattan_data.drop(['Postal Code','Borough','Neighborhood'],1)
#kmeans = KMeans(n_clusters = k,random_state=0).fit(manhattan_clustering)
#kmeans.labels_
#manhattan_data.insert(0, 'Cluster Labels', kmeans.labels_)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Gym,Sandwich Place,Discount Store,Coffee Shop,Yoga Studio,Video Game Store,Seafood Restaurant,Tennis Stadium,Donut Shop,Supplement Shop
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Salon / Barbershop,Spa,Optical Shop,Dessert Shop,Greek Restaurant,Sandwich Place
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Latin American Restaurant,Bank,Italian Restaurant,Tapas Restaurant,Gym
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Café,Lounge,Restaurant,Pizza Place,Deli / Bodega,Spanish Restaurant,Wine Bar,Frozen Yogurt Shop,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Mexican Restaurant,Café,Coffee Shop,Deli / Bodega,Yoga Studio,Park,Indian Restaurant,Latin American Restaurant,Cocktail Bar


In [106]:
address = 'Manhattan'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        #color = rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters